In [232]:
from datasets import basic_entities
from glob import glob
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display # to display images
import os

%matplotlib inline

lines = {
    'IM05': [
        # Bottom
        ((600, 400), (610, 719)),
        # Right
        ((800, 300), (1200, 250)),
        # Left
        ((0, 250), (350, 380))
    ],
    'IM02': [
        # Bottom left
        ((320, 400), (500, 450)),
        # Bottom right
        ((550, 400), (900, 470)),
        #Middle
        ((400, 200), (850, 260)),
        #Top
        ((450, 50), (900, 100))
        
    ],
    'IM01': [
        # Bottom
        ((150, 700), (750, 450)),
        # Top
        ((60, 220), (500, 50))
    ],
    'IM03': [
        # Middle all
        ((600, 550), (640, 250)),
        # Single line, top
        ((800, 380), (810, 300)),
        # Small line, right
        ((1000, 390), (980, 600))
    ],
    'IM04': [
        # Top centre
        ((740, 160), (830, 90)),
        # Left centre
        ((750, 350), (920, 480)),
        # Top
        ((200, 200), (520, 0)),
        # Left
        ((20, 250), (300, 720))
    ]
}

In [233]:
import scipy.io
import pickle
import numpy as np

all_crosses = {}
for image_dir in glob('data/TUBCrowdFlow/images/IM??'):
    base = os.path.basename(image_dir)
    frame_files = glob('{}/*'.format(image_dir))
    frame_files.sort()
    
    if base not in lines:
        print("Skipped {}".format(base))
        continue
        
    track_path = '{}/PersonTracks.pb'.format(image_dir.replace('images', 'gt_trajectories'))
    ret = pickle.load(open(track_path, 'rb'))
    
    print("Calculate for {}".format(base))
    
    all_crosses[base] = {}
    for o, line in enumerate(lines[base]):
        
        crosses = [[], []]
        for i, pedestrian in enumerate(ret['GT_Trajectories']):

            arr = np.array(pedestrian)
            arr = arr[:,[1, 0]]

            base_point = np.array(line[0])
            vector = np.array(line[1])
            norm_vector = vector - base_point
            norm_arr = arr - base_point

            # Project to check which points fall inside the line
            upper_proj = np.dot(norm_arr, norm_vector)
            lower_proj = np.linalg.norm(norm_vector)**2
            proj = upper_proj / lower_proj
            inside = np.array([proj >= 0, proj <= 1]).all(axis=0)

            # Are you on the right side of the line
            check = (norm_vector[0]*norm_arr[:, 1] - norm_vector[1]*norm_arr[:, 0]) > 0;

            # If it didn't cross, then no need to check further
            if np.all(check == check[0]):
                continue

            check2 = np.roll(check, 1)
            together = check[1:] != check2[1:]
            poses = np.where(together == True)[0]
            
            pos = None
            for pos_i in poses:
                if inside[pos_i] == False:
                    continue
                
                # Some weird start displacements are filtered out this way
                norm = np.linalg.norm([arr[pos_i,0]-arr[pos_i+1,0], arr[pos_i,1]-arr[pos_i+1,1]])
                if norm > 20:
                    continue
                
                pos = pos_i
            
            if pos == None:
                continue
            
            
            crossing_frame = int(ret['GT_StartPoints'][i][0] + pos + 2)
            crosses[int(check[pos+1])].append(crossing_frame)
        
        print("Finished line", len(crosses[0]), len(crosses[1])) # 0 == left to right, 1 = right to left
        all_crosses[base][o] = crosses

Calculate for IM05
Finished line 316 0
Finished line 397 0
Finished line 0 410
Calculate for IM04
Finished line 93 0
Finished line 0 54
Finished line 63 0
Finished line 0 90
Calculate for IM02
Finished line 0 55
Finished line 0 47
Finished line 0 113
Finished line 0 122
Calculate for IM01
Finished line 8 84
Finished line 8 84
Calculate for IM03
Finished line 80 101
Finished line 0 28
Finished line 71 72


In [ ]:
def video_into_samples(frames, begin, end, length, overlap):
    

In [234]:
# Split every video in test and validation
test_part = 0.5
val_length = 50
val_overlap = 50 # So 50%

all_samples = {}

for image_dir in glob('data/TUBCrowdFlow/images/IM??'):
    base = os.path.basename(image_dir)
    frame_files = glob('{}/*'.format(image_dir))
    frame_files.sort()
    count_frames = len(frame_files)
    
    if base not in all_crosses:
        continue
    
    # The frames that people cross for this current video
    video_crosses = all_crosses[base]
    video_samples = []
    
    for line_i in video_crosses:
        line_crosses = video_crosses[line_i]

        test_end_frame = int(count_frames * test_part)
        # Extract from the validation part, take smaller parts as evaluation sample
        for o in range(test_end_frame, count_frames-val_overlap+1, val_overlap):
            start_sample = o
            end_sample = min(o+val_length, count_frames)
            # Check all the original frame jumps and take only the ones with this sample
            check_line_crosses = []
            for line_cross in line_crosses:
                valid_line_cross = []
                for frame_jump in line_cross:
                    if frame_jump >= start_sample and frame_jump < end_sample:
                        valid_line_cross.append(frame_jump)
                check_line_crosses.append(valid_line_cross)
            
            video_samples.append([start_sample, end_sample, lines[base][line_i], check_line_crosses])
    
    all_samples[base] = video_samples

In [235]:
for video in all_samples:
    print('{}: {} samples'.format(video, len(all_samples[video])))

IM05: 12 samples
IM04: 12 samples
IM02: 12 samples
IM01: 6 samples
IM03: 6 samples


In [239]:
# Display and save all the lines drawn in the video's
for image_dir in glob('data/TUBCrowdFlow/images/IM??'):
    base = os.path.basename(image_dir)
    frame_files = glob('{}/*'.format(image_dir))
    frame_files.sort()
    
    first_frame = frame_files[0]
    pil_image = Image.open(first_frame)
    draw = ImageDraw.Draw(pil_image)
    
    video_lines = lines[base]
    for line in video_lines:
        draw.line(line, width=7, fill=(255,0,0))

    pil_image.save('results/tub_lines/{}.png'.format(base))